In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('PartD_Prescriber_PUF_NPI_17.txt', sep='\t', engine='python')
data.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,...,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,4.5148


## Question 1

In [14]:
data['bene_count'].mean(), data.loc[~data['bene_count'].isna(), 'bene_count'].mean()

(158.3494585173676, 158.3494585173676)

## Question 2: the length of the average prescription 

In [15]:
data['avg_length']=data.total_day_supply/data.total_claim_count
data['avg_length'].median()

29.7125748502994

## Question 3

In [30]:
spec=data[data.brand_suppress_flag.isna()].groupby(by=['specialty_description']).sum().reset_index()
spec['brand_fraction']=spec.brand_claim_count/spec.total_claim_count
spec.loc[spec.total_claim_count>=1000, 'brand_fraction'].std()


0.09664998667912822

## Question 4

In [41]:
state=data.groupby(by=['nppes_provider_state']).sum()[['opioid_bene_count', 'antibiotic_bene_count']].reset_index()
state['ratio']=state.opioid_bene_count/state.antibiotic_bene_count
state.ratio.max()-state.ratio.min()


0.6331097230267027

## Question 5

In [45]:
data['Older65']=data.total_claim_count_ge65/data.total_claim_count
data['lowIncome']=data.lis_claim_count/data.total_claim_count
print("The pearson correlation coeff is", data.Older65.corr(data.lowIncome, method='pearson'))

The pearson correlation coeff is -0.6389319595676912


## Question 6

In [52]:
data['Op_length']=data.opioid_day_supply/data.opioid_claim_count
highOp=data.groupby(by=['nppes_provider_state', 'specialty_description']).aggregate({'npi': 'count', 'Op_length':'mean'}).reset_index()
highOp=highOp[highOp['npi']>=100]
highOp.set_index('specialty_description', inplace=True)

In [58]:
highOpAll=data.groupby(by=['specialty_description']).aggregate({'Op_length':'mean'}).reset_index().set_index('specialty_description')
highOpAll.rename(columns={'Op_length':'Op_length_all'}, inplace=True)
highOp=highOp.join(highOpAll)


0

In [61]:
highOp['highOp_ratio']=highOp.Op_length/highOp.Op_length_all
highOp['highOp_ratio'].max()

2.304597636251375

## Question 7

In [63]:
fields = ['npi', 'total_claim_count', 'total_day_supply', 'total_drug_cost']
data16 = pd.read_csv('PartD_Prescriber_PUF_NPI_16.txt', sep='\t', engine='python', usecols=fields)

merged=data[fields].merge(data16, on='npi', how='left', suffixes=('_17', '_16'))
merged.head()

,npi,total_claim_count_17,total_day_supply_17,total_drug_cost_17,total_claim_count_16,total_drug_cost_16,total_day_supply_16
0,1003000126,677,14788,32639.57,545.0,33618.18,12120.0
1,1003000142,1946,58605,140189.01,1733.0,139695.81,51314.0
2,1003000167,55,430,302.01,49.0,267.81,509.0
3,1003000175,18,150,113.81,NaN,NaN,NaN
4,1003000282,90,2681,7561.21,146.0,4587.11,5939.0


In [69]:
for year in [16,17]:
    merged['avg_cost_day_'+str(year)]=merged['total_drug_cost_'+str(year)]/merged['total_day_supply_'+str(year)]
merged['inflation']=merged.avg_cost_day_17/merged.avg_cost_day_16-1
merged.inflation.mean()

0.2274706790254255

## Question 8

In [84]:
fields = ['npi', 'specialty_description']
data16 = pd.read_csv('PartD_Prescriber_PUF_NPI_16.txt', sep='\t', engine='python', usecols=fields)
merged=data[fields].merge(data16, on='npi', how='left', suffixes=('_17', '_16'))
merged=merged[(~merged.specialty_description_16.isna()) & (~merged.specialty_description_17.isna())]
merged['change']=np.where(merged.specialty_description_16 !=merged.specialty_description_17 ,1, 0)
merged.describe()

,npi,change
count,1.054559e+06,1.054559e+06
mean,1.499864e+09,6.754672e-02
std,2.879302e+08,2.509666e-01
min,1.003000e+09,0.000000e+00
25%,1.245692e+09,0.000000e+00
50%,1.497968e+09,0.000000e+00
75%,1.750300e+09,0.000000e+00
max,1.993000e+09,1.000000e+00


In [85]:
group=merged.groupby(by='specialty_description_16').agg({'specialty_description_16': 'count', 'change':'sum'})
group['change_ratio']=group.change/group.specialty_description_16
group=group[group['specialty_description_16']>=1000]
group.sort_values('change_ratio', ascending=False, inplace=True )
group.head()

,specialty_description_16,change,change_ratio
specialty_description_16,,,
Obstetrics/Gynecology,4689,4689,1.000000
Cardiovascular Disease (Cardiology),20079,20079,1.000000
Clinical Cardiatric Electrophysiology,1877,1877,1.000000
Oral Surgery (Dentists only),5418,5418,1.000000
Student in an Organized Health Care Education/Training Program,45388,13172,0.290209


In [83]:
# Second Method
group16=merged.groupby(by=['specialty_description_16'])['specialty_description_16'].count()
group17=merged.groupby(by=['specialty_description_17'])['specialty_description_17'].count()
group16=pd.concat([group16, group17], axis=1, join='outer')
group16['change_ratio']=(group16.specialty_description_16-group16.specialty_description_17)/group16.specialty_description_16
group16.sort_values('change_ratio', ascending=False, inplace=True )
group16[group16.specialty_description_16>=1000].head()


,specialty_description_16,specialty_description_17,change_ratio
Student in an Organized Health Care Education/Training Program,45388.0,32628.0,0.281132
Cardiac Surgery,1060.0,1020.0,0.037736
Certified Clinical Nurse Specialist,2478.0,2409.0,0.027845
General Practice,9425.0,9317.0,0.011459
Internal Medicine,126116.0,124698.0,0.011244
